In [8]:
%load_ext autoreload
%autoreload 2
from collections import Counter
from pathlib import Path

import fasttext
import spacy

Path.ls = lambda x: list(x.iterdir())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Train Model

In [9]:
# !python -m pip install spacy
# !python -m spacy download en_core_web_sm

In [10]:
def get_raw_text(filepath: str):
    with Path(filepath).open("r") as f:
        plaintext = f.read()
    return str(plaintext)

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 1.1*1341865 # allow greater memory consumption

plaintext = get_raw_text("airbnb.txt")
custom_stop_words = ["$", "\n", "common"] 
# "Class", "2020", "2019", "million", "shares", "Company", "business"]
for word in custom_stop_words:
    nlp.Defaults.stop_words.add(word)
%time doc = nlp(plaintext)

CPU times: user 28.6 s, sys: 6.42 s, total: 35 s
Wall time: 36.7 s


In [11]:
words = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_digit and len(token)>1]
words = " ".join(words)
with open("stripped_airbnb.txt", "w") as f:
    f.write(words)
model = fasttext.train_unsupervised("stripped_airbnb.txt")
model.save_model("lemma_stripped_airbnb.bin")

In [12]:
model = fasttext.load_model("lemma_stripped_airbnb.bin")